In [78]:
import pandas as pd
import os
import sys
parent_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(os.path.abspath(parent_path))
from constants import HAWAJEZ ,INFORMATION_WORDS
import random

In [157]:
data = pd.read_csv('../data/cleaned_transformed_data.csv')


In [158]:
# make the dataframe from full_text column only
data = data[['full_text']]

In [159]:
def replace_word_with_hawajez(text):
    """Replace a word in the given text with a random word from the Hawajez list."""
    for word in HAWAJEZ:
        if word in text:
            new_word = random.choice(HAWAJEZ)
            return text.replace(word, new_word)
    return text

In [160]:

def replace_status_with_new_status(text):
    """Replace a word in the given text with a random word from the Hawajez list."""
    for  info in INFORMATION_WORDS:
        if info in text:
            new_info = random.choice(INFORMATION_WORDS)
            return text.replace(info, new_info)
    return text

In [161]:
import concurrent.futures
import warnings

warnings.filterwarnings('ignore')


new_data = pd.DataFrame(columns=["full_text"])
def process_row(row):
    new_rows = []
    for i in range(0, random.randint(30,50)):
        new_text = replace_word_with_hawajez(row["full_text"])
        new_text_status = replace_status_with_new_status(new_text)
        if new_text:
            new_rows.append({"full_text": new_text_status})
    return new_rows

# Use ThreadPoolExecutor to run the loop in parallel threads
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit a thread for each row in the data
    future_to_row = {executor.submit(process_row, row): row for _, row in data.iterrows()}
    # Collect the results as they become available
    for future in concurrent.futures.as_completed(future_to_row):
        new_rows = future.result()
        # Append the new rows to the new data
        if new_rows:
            new_data = new_data.append(new_rows, ignore_index=True)

pd.set_option('display.max_colwidth', None)
new_data.shape
# should i concatinat the new data with the old data ? 
# a: yes
# show me how ?
data = pd.concat([data, new_data], ignore_index=True)

In [162]:
data.shape

(805422, 1)

In [165]:
# check if theres any null values
data.isnull().sum()

full_text    0
dtype: int64

In [176]:
# suffle the rows
data = data.sample(frac=1).reset_index(drop=True)

In [177]:
data.iloc[50000:50010]

,full_text
50000,حاليا دوار جبع علي سيارتين شرطه وفي سحب سيارات
50001,من باب رامي ليفي لالمربعه مغلق واقفه
50002,نظيف الكنتنر فل الفل
50003,دي سي؟|||فش اشي
50004,المربعه مفتوحة|||اا مفتوحة
50005,الفندق|||مستوطنة
50006,صرة للداخل شو وضعه|||تفتيشة
50007,كل الطرق نظيف فقط حاجز حواره مسكر للداخل والخارج طريق الكونتينر فاتحه للداخل والخارج
50008,عراق بورين للخارج من نابلس تفتيش دقيق وازمة للداخل فاتح البديل عن حاجز عراق بورين عن معبر عورتاصرة فاتحة بالاتجاهين لهاي اللحظة دير شرف فاتح بالاتجاهين لهاي اللحظةخط يتسهار فاتح المربعة حاجز وتفتيش عراق بورين سكروها بالتراببيت فوريك تفتيش دقيق وازمةخط رام الله كامل فاتحبيت ايل فاتحخط عراق بورين البلد فاتح مع تواجد للجيش
50009,في اشي على الطنيب ولا خرجت|||لا خرجت


In [179]:
data = data[['full_text']]
data.to_csv('../data/more_data.csv', index=False)